In [1]:
import os
from PIL import Image

import time
import numpy as np
import tensorflow as tf
import pandas as pd


from user import User
from average import Average
from tensorflow import keras

from graphing import *
from file_related import *
from inits import *
from trainers import *
SEED = 0

# import os
# os.environ['PYTHONHASHSEED']=str(SEED)
# np.random.seed(SEED)
# import random
# random.seed(SEED)
# tf.set_random_seed(SEED)
# could need to force keras to not use parallelism, see documentation

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:

# https://www.kaggle.com/prasunroy/natural-images
data_dir = "../testing/data/natural_images/"

CLASSES, files = map_images(data_dir)
averaging_methods = [Average.all,Average.std_dev,Average.weighted_avg]


In [ ]:
def train(model, toy=False):
        """
        Train the model
        """
        if toy:
            epochs=3
            steps_per_epoch=20
            validation_steps=2
        else:
            epochs=30
#             steps_per_epoch=100
            steps_per_epoch=30
#             steps_per_epoch=10
            validation_steps=5
            
        # We'll stop training if no improvement after some epochs
        earlystopper = EarlyStopping(monitor='val_acc', patience=10, verbose=1)
        reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
        # Save the best model during the traning
        checkpointer = ModelCheckpoint('best_model1.h5'
                                        ,monitor='val_acc'
                                        ,verbose=1
                                        ,save_best_only=True
                                        ,save_weights_only=True)
        # Train
        training = model.fit(x_train, y_train
                                ,epochs=epochs
                                ,steps_per_epoch=steps_per_epoch
                                ,validation_data=(x_val, y_val)
                                ,validation_steps=validation_steps
                                ,callbacks=[earlystopper, checkpointer, reduce_lr], verbose=3)
        # Get the best saved weights
        model.load_weights('best_model1.h5')
        return training

In [ ]:
ROUNDS = 32 # after about 30-40, it seems to settle down with epochs being 16
EPOCHS = 16
# MAJORITY_SPLIT = 0.7
OUT_PATH = "../out/images"

In [ ]:
for MAJORITY_SPLIT in [1/8, 0.5, 0.7, 0.99]: #
    
    print(f"\n\n\n\n\n\n========={MAJORITY_SPLIT}========\n\n\n\n\n")
    # forces tensorfor to the a particular device to run things
    #      # return a dictionary of users with data
    print("Regular")

    users_all = init_users_image(files, averaging_methods, majority_split = MAJORITY_SPLIT, seed = SEED)
    train_fed(users = users_all, rounds = ROUNDS,
              epochs = EPOCHS,
              strat = "central",
              train_user_verbose = False,
              train_user_verbose_evaluate = False,
              train_user_verbose_fit = False,
              averaging_method = Average.all,
              averaging_post = True,
              averaging_metric = "accuracy")


    averaging_methods = [Average.std_dev]
    # return a dictionary of users with data
    print("avg - stddev")

    users_std_dev = init_users_image(files, averaging_methods,majority_split = MAJORITY_SPLIT, seed = SEED)
    train_fed(users = users_std_dev, rounds = ROUNDS,
              epochs = EPOCHS,
              strat = "central",
              train_user_verbose = False,
              train_user_verbose_evaluate = False,
              train_user_verbose_fit = False,
              averaging_method =  Average.std_dev,
             averaging_post = True,
              averaging_metric = "accuracy")

    # return a dictionary of users with data
    users_std_dev_p2p = init_users_image(files, averaging_methods,majority_split = MAJORITY_SPLIT, seed = SEED)
    train_fed(users = users_std_dev_p2p, rounds = ROUNDS,
              epochs = EPOCHS,
              strat = "personalised",
              train_user_verbose = False,
              train_user_verbose_evaluate = False,
              train_user_verbose_fit = False,
              averaging_post = True,
              averaging_metric = "accuracy")

    averaging_methods = [Average.weighted_avg]
    print("Weighted")
    # return a dictionary of users with data
    users_weighted_avg = init_users_image(files, averaging_methods,majority_split = MAJORITY_SPLIT, seed = SEED)
    train_fed(users = users_weighted_avg, rounds = ROUNDS,
              epochs = EPOCHS,
              strat = "central",
              train_user_verbose = False,
              train_user_verbose_evaluate = False,
              train_user_verbose_fit = False,
              averaging_method = Average.weighted_avg,
              averaging_post = True,
              averaging_metric = "accuracy")

    # return a dictionary of users with data
    users_weighted_avg_p2p = init_users_image(files, averaging_methods,majority_split = MAJORITY_SPLIT, seed = SEED)
    train_fed(users = users_weighted_avg_p2p, rounds = ROUNDS,
              epochs = EPOCHS,
              strat = "personalised",
              train_user_verbose = False,
              train_user_verbose_evaluate = False,
              train_user_verbose_fit = False,
              averaging_post = True,
              averaging_metric = "accuracy")
    print("Local only")
    users_local_only = init_users_image(files, averaging_methods,majority_split = MAJORITY_SPLIT, seed = SEED)
    train_fed(users = users_local_only, rounds = ROUNDS,
              epochs = EPOCHS,

              strat = "local_trainings_only",
              train_user_verbose = False,
              train_user_verbose_evaluate = False,
              train_user_verbose_fit = False,
              averaging_post = True,
              averaging_metric = "accuracy")

    print("Global")

    global_user = init_users_image(files,averaging_methods,majority_split = MAJORITY_SPLIT, seed = SEED, return_global_user=True)
    # print(global_user.get_train_data().dtype)
    # global_user.get_model().summary()
    global_user.train(
        epochs = ROUNDS*EPOCHS,
        verbose_fit = False,
        verbose_evaluate = False
    )

    ignore_first_n = 0
    final_values = False
    for GRAPHING_METRIC in ["accuracy", "loss"]:
        round_summary_df = pd.DataFrame(columns=['Strategy','Position', 'Round', 'Average', 'Standard Deviation', 'Minimum', 'Maximum'])
        user_summary_df = pd.DataFrame(columns=['Strategy', 'Position', 'User', 'Average', 'Standard Deviation', 'Minimum', 'Maximum', 'Final Value'])
        print("All")

        print("For central")


        df_avg_round_stats_acc = avg_round_stats(users_all, 
                                                 ignore_first_n = ignore_first_n, 
                                                 min_max_fill = True, 
                                                 metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"centralround-{MAJORITY_SPLIT}",
                                                 final_values = final_values)
        df_avg_user_stats_acc = avg_user_stats(users_all, 
                                               ignore_first_n = ignore_first_n, 
                                               min_max_fill = True, 
                                               metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"centraluser-{MAJORITY_SPLIT}",
                                               final_values = final_values)
        last_2 = df_avg_round_stats_acc[-2:]
        last_2["Strategy"] = "centralround"
        round_summary_df = round_summary_df.append(last_2).reset_index(drop=True)
        df_avg_user_stats_acc["Strategy"] = "centraluser"
        user_summary_df = user_summary_df.append(df_avg_user_stats_acc).reset_index(drop=True)
        print("Std deviation")
        print("For central")


        df_avg_round_stats_acc = avg_round_stats(users_std_dev, 
                                                 ignore_first_n = ignore_first_n, 
                                                 min_max_fill = True, 
                                                 metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"stdround-{MAJORITY_SPLIT}",
                                                 final_values = final_values)
        df_avg_user_stats_acc = avg_user_stats(users_std_dev, 
                                               ignore_first_n = ignore_first_n, 
                                               min_max_fill = True, 
                                               metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"stduser-{MAJORITY_SPLIT}",
                                               final_values = final_values)
        last_2 = df_avg_round_stats_acc[-2:]
        last_2["Strategy"] = "stdround"
        round_summary_df = round_summary_df.append(last_2).reset_index(drop=True)
        df_avg_user_stats_acc["Strategy"] = "stduser"
        user_summary_df = user_summary_df.append(df_avg_user_stats_acc).reset_index(drop=True)
        
        print("For p2p")
        
        df_avg_round_stats_acc = avg_round_stats(users_std_dev_p2p, 
                                                 ignore_first_n = ignore_first_n, 
                                                 min_max_fill = True, 
                                                 metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"stdround_p2p-{MAJORITY_SPLIT}",
                                                 final_values = final_values)
        df_avg_user_stats_acc = avg_user_stats(users_std_dev_p2p, 
                                               ignore_first_n = ignore_first_n, 
                                               min_max_fill = True, 
                                               metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"stduser_p2p-{MAJORITY_SPLIT}",
                                               final_values = final_values)
        last_2 = df_avg_round_stats_acc[-2:]
        last_2["Strategy"] = "stdround_p2p"
        round_summary_df = round_summary_df.append(last_2).reset_index(drop=True)
        df_avg_user_stats_acc["Strategy"] = "stduser_p2p"
        user_summary_df = user_summary_df.append(df_avg_user_stats_acc).reset_index(drop=True)


        print("Weighted average")

        print("For central")


        df_avg_round_stats_acc = avg_round_stats(users_weighted_avg, 
                                                 ignore_first_n = ignore_first_n, 
                                                 min_max_fill = True, 
                                                 metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"weightround-{MAJORITY_SPLIT}",
                                                 final_values = final_values)
        df_avg_user_stats_acc = avg_user_stats(users_weighted_avg, 
                                               ignore_first_n = ignore_first_n, 
                                               min_max_fill = True, 
                                               metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"weightuser-{MAJORITY_SPLIT}",
                                               final_values = final_values)
        last_2 = df_avg_round_stats_acc[-2:]
        last_2["Strategy"] = "weightround"
        round_summary_df = round_summary_df.append(last_2).reset_index(drop=True)
        df_avg_user_stats_acc["Strategy"] = "weightuser"
        user_summary_df = user_summary_df.append(df_avg_user_stats_acc).reset_index(drop=True)

        print("For p2p")


        df_avg_round_stats_acc = avg_round_stats(users_weighted_avg_p2p, 
                                                 ignore_first_n = ignore_first_n, 
                                                 min_max_fill = True, 
                                                 metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"weightround_p2p-{MAJORITY_SPLIT}",
                                                 final_values = final_values)
        df_avg_user_stats_acc = avg_user_stats(users_weighted_avg_p2p, 
                                               ignore_first_n = ignore_first_n, 
                                               min_max_fill = True, 
                                               metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"weightuser_p2p-{MAJORITY_SPLIT}",
                                               final_values = final_values)

        last_2 = df_avg_round_stats_acc[-2:]
        last_2["Strategy"] = "weightround_p2p"
        round_summary_df = round_summary_df.append(last_2).reset_index(drop=True)
        df_avg_user_stats_acc["Strategy"] = "weightuser_p2p"
        user_summary_df = user_summary_df.append(df_avg_user_stats_acc).reset_index(drop=True)


        print("Local training only")


        df_avg_round_stats_acc = avg_round_stats(users_local_only, 
                                                 ignore_first_n = ignore_first_n, 
                                                 min_max_fill = True, 
                                                 metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"localround-{MAJORITY_SPLIT}",
                                                 final_values = final_values)
        df_avg_user_stats_acc = avg_user_stats(users_local_only, 
                                               ignore_first_n = ignore_first_n, 
                                               min_max_fill = True, 
                                               metric = GRAPHING_METRIC, path= OUT_PATH, save_as = f"localuser-{MAJORITY_SPLIT}",
                                               final_values = final_values)
        # draw_graphs(users_all_p2p[2])
        last_2 = df_avg_round_stats_acc[-2:]
        last_2["Strategy"] = "localround"
        round_summary_df = round_summary_df.append(last_2).reset_index(drop=True)
        df_avg_user_stats_acc["Strategy"] = "localuser"
        user_summary_df = user_summary_df.append(df_avg_user_stats_acc).reset_index(drop=True)

        print(round_summary_df)
        round_summary_df.to_csv(os.path.join(OUT_PATH,f'summary-rounds-{GRAPHING_METRIC}-{MAJORITY_SPLIT}.csv')) 
        user_summary_df.to_csv(os.path.join(OUT_PATH,f'summary-users-{GRAPHING_METRIC}-{MAJORITY_SPLIT}.csv')) 

        draw_graphs(global_user, path= OUT_PATH, save_as=f"global_{MAJORITY_SPLIT}")

        with open(f"{OUT_PATH}{GRAPHING_METRIC}-{MAJORITY_SPLIT}.csv", "w") as out:
            strats = [users_local_only, users_all, users_std_dev, users_std_dev_p2p, users_weighted_avg, users_weighted_avg_p2p]
            strats_names = ["Local only", "All (traditional)", "Std dev Central", "Std dev P2P", "Weighted avg Central", "Weighted avg P2P"]
            g_weights = global_user.get_weights()
            out.write("Testing user models (post fit) on global model aggregate data,\n")
            e = global_user.evaluate(verbose=False)
            out.write(f"{GRAPHING_METRIC}-globaluser,{e[0]},{e[1]},\n")
        #     print(global_user.evaluate(verbose=False))

            for i, strat in enumerate(strats):
        #         print(strats_names[i])
                out.write(strats_names[i]+",\n")
                for user in strat.values():
                    global_user.set_weights(user.get_weights())
                    e = global_user.evaluate(verbose=False)
                    out.write(f"{GRAPHING_METRIC}-user-{user.get_id()}:,{e[0]},{e[1]},\n")
        #             print(f"{GRAPHING_METRIC}-{user.get_id()}: {e}")


In [ ]:
# # user = users_local_only[4]
# for i, user in enumerate(users_all.values()):
#     print(f"{GRAPHING_METRIC}-{CLASSES[i]:<10} -> {user.get_id()}: {np.unique(user.get_train_class(), return_counts=True)}")
    
# print(f"{GRAPHING_METRIC}-{' ':<10} -> {global_user.get_id()}: {np.unique(global_user.get_train_class(), return_counts=True)}")
# print(f"{GRAPHING_METRIC}-{' ':<10} -> {global_user.get_id()}: {np.unique(global_user.get_test_class(), return_counts=True)}")
# print(f"{GRAPHING_METRIC}-{' ':<10} -> {global_user.get_id()}: {np.unique(global_user.get_val_class(), return_counts=True)}")

# # print(np.equal(users_all[1].get_test_class(),users_weighted_avg[1].get_test_class()).all())

In [ ]:
df_avg_user_stats_acc.columns
